# 1. Load data
## 1.1 Ground truth

In [35]:
import os
import csv
import json

import matplotlib.pyplot as plt

In [36]:
frames_true_files = os.listdir('ground_truth/true')
frames_false_files = os.listdir('ground_truth/false')

In [37]:
frames_true = [list(map(int, x.split('.')[0].split('_'))) for x in frames_true_files]
frames_false = [list(map(int, x.split('.')[0].split('_'))) for x in frames_false_files]

In [38]:
data = dict()
for x in frames_true:
    data[f'{x[0]}_{x[1]}'] = {'ground_truth': True}
for x in frames_false:
    data[f'{x[0]}_{x[1]}'] = {'ground_truth': False}
# print(data)

## 1.2 Dane z czujnika z normalizacją min-max

In [39]:
with open('min_max.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    for id_, value in your_list:
        if id_ in data:
            data[id_]['min_max'] = value

## 1.2 Dane z czujnika z normalizacją min-max

In [40]:
with open('z_score.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    for id_, value in your_list:
        if id_ in data:
            data[id_]['z_score'] = value

## 1.3 Wyniki inferencji

In [41]:
with open('inference.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    for id_, value in your_list:
        if id_ in data:
            data[id_]['predicted'] = value

# 2. Sortowanie danych

## 2.1 Utworzenie listy, pominiędzie ramek niezawierajacych wszystkich informacji

In [42]:
data_list = []
for id_, x in data.items():
    if 'z_score' in x and 'min_max' in x and 'predicted' in x and 'ground_truth' in x:
        id_split = id_.split('_')
        video_id = int(id_split[0])
        frame_id = int(id_split[1])
        data_list.append((id_, video_id, frame_id,  float(x['z_score']), float(x['min_max']), x['predicted'] == "True", x['ground_truth']))

print(len(data_list))

15633


In [43]:
import functools


def extract_ids(val):
    key = val[0]
    key_split = key.split('_')
    video_id = int(key_split[0])
    frame_id = int(key_split[1])
    return video_id, frame_id


def compare_id(x, y):
    v_x, f_x = extract_ids(x)
    v_y, f_y = extract_ids(y)
    v_x *= 1000_000
    v_y *= 1000_000
    return (v_x + f_x) - (v_y - f_y)
    # if v_x != v_y:
    #     return v_x > v_y
    # else:
    #     return f_x >= f_y


data_sorted = sorted(data_list, key=lambda key: (key[1], key[2]))
print(data_sorted[:100])

[('0_109', 0, 109, -1.1573821046220343, 0.0, False, False), ('0_114', 0, 114, -1.1573821046220343, 0.0, False, False), ('0_124', 0, 124, -1.1573821046220343, 0.0, False, False), ('0_134', 0, 134, -1.1573821046220343, 0.0, False, False), ('0_139', 0, 139, -1.1573821046220343, 0.0, False, False), ('0_149', 0, 149, -1.1573821046220343, 0.0, False, False), ('0_174', 0, 174, -1.0701058442104696, 0.013333333333333336, False, False), ('0_199', 0, 199, -1.0701058442104696, 0.013333333333333336, False, False), ('0_204', 0, 204, -0.9828295837989051, 0.026666666666666672, False, False), ('0_219', 0, 219, -0.9828295837989051, 0.026666666666666672, False, False), ('0_224', 0, 224, -0.8955533233873404, 0.04, False, False), ('0_229', 0, 229, -0.8955533233873404, 0.04, False, False), ('0_244', 0, 244, -0.8955533233873404, 0.04, False, False), ('0_249', 0, 249, -0.8955533233873404, 0.04, False, False), ('0_254', 0, 254, -0.8955533233873404, 0.04, False, False), ('0_269', 0, 269, -0.8082770629757757, 0.

In [44]:
f = open(f"data.csv", 'w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(['id', 'video_id', 'frame_id', 'sensor_z_score', 'sensor_min_max', 'predicted', 'ground_truth'])
writer.writerows(data_sorted)
f.close()

In [45]:
f = open(f"data_village.csv", 'w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(['id', 'video_id', 'frame_id', 'sensor_z_score', 'sensor_min_max', 'predicted', 'ground_truth'])
for x in data_sorted:
    if x[1] < 2:
        writer.writerow(x)
f.close()

In [46]:
f = open(f"data_city.csv", 'w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(['id', 'video_id', 'frame_id', 'sensor_z_score', 'sensor_min_max', 'predicted', 'ground_truth'])
for x in data_sorted:
    if x[1] >= 2:
        writer.writerow(x)
f.close()